In [1]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.runnables import RunnablePassthrough,RunnableLambda, Runnable, RunnableParallel,RunnableConfig
from langchain_core.messages import AIMessage
from dotenv import load_dotenv,find_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate,SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from operator import itemgetter
from langchain.embeddings import SentenceTransformerEmbeddings
import json
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS,Chroma
from operator import itemgetter
import time
import grandalf

C:\Users\MSI 1\.conda\envs\oaivenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(find_dotenv("D:\LLM Courses\Master Langchain Udemy\.env"))

True

In [3]:
llm=ChatGoogleGenerativeAI(model="gemini-1.5-flash")
# llm=ChatOpenAI(model="gpt-3.5-turbo")

In [4]:
prompt=ChatPromptTemplate.from_template(
                        template="Tell me a joke about {topic}")

In [41]:
chain=prompt|llm|StrOutputParser()
chain.invoke(input={'topic':"Bears"})

'Why did the bear say no to dessert?\n\nBecause he was stuffed! 🐻 🍰 \n'

In [38]:
chainRitzStyle=RunnablePassthrough.assign(topic=itemgetter("question")|prompt).assign(result=itemgetter('topic')|llm).assign(output=itemgetter('result')|StrOutputParser())|RunnableLambda(lambda x:x['output'])

In [39]:
chainRitzStyle.invoke(input={'question':"Bears"})

'Why did the bear say no to dessert?\n\nBecause he was already stuffed! 🐻 🍨 \n'

<h3> Coercion</h3>

In [42]:
analysisPrompt=ChatPromptTemplate.from_template(
    template="Is this a funny joke? {joke}"
)

In [46]:
combinedChain={"joke":chainRitzStyle}|analysisPrompt|llm|StrOutputParser()

In [47]:
combinedChain.invoke(input={'question':"Bears"})

'Yes, that\'s a funny joke! It\'s a classic "dad joke" that relies on a pun. \n\nHere\'s why it works:\n\n* **Wordplay:**  The joke plays on the double meaning of "stuffed." It can mean "full of food" and it can also mean "filled with stuffing," which is something you might find inside a Thanksgiving turkey. \n* **Unexpected Twist:**  The listener expects a more traditional joke about bears, but the punchline uses a clever twist on the word "stuffed" to create a humorous surprise.\n* **Simple and Relatable:**  The joke is easy to understand, even for young children. It also relates to a common experience of feeling full after a meal.\n\nOverall, it\'s a lighthearted and silly joke that\'s sure to get a chuckle. \n'

<h3> Using the Pipe Method</h3>

In [50]:
composedChainWithPipe=RunnablePassthrough.assign(joke=chainRitzStyle).pipe(analysisPrompt).pipe(llm).pipe(StrOutputParser())

In [51]:
composedChainWithPipe.invoke(input={'question':"Bears"})

'Yes, that\'s a funny joke! It\'s a classic pun using the word "stuffed" to refer to both being full after eating and the act of filling a stuffed animal. \n\nHere\'s why it works:\n\n* **Unexpected twist:** The setup leads you to expect a reason related to bears, but the punchline is a play on words.\n* **Simple and relatable:**  Everyone can understand the feeling of being "stuffed" after a big meal.\n* **Visual humor:** The image of a full bear refusing dessert is amusing.\n\nOverall, it\'s a short, clever, and easily understood joke that makes people smile. \n'